In [2]:
'''
Question 2 : 
Point the camera to a chessboard pattern or any known set of reference points that lie on the
same plane. Capture a series of 10 images by changing the orientation of the camera in each
iteration. Select any 1 image, and using the image formation pipeline equation, set up the linear
equations in matrix form and solve for intrinsic and extrinsic parameters (extrinsic for that
particular orientation). You will need to make measurements of the actual 3D world points, and
mark pixel coordinates. Once you compute the Rotation matrix, you also need to compute the
angles of rotation along each axis. Choose your order of rotation based on your experimentation
setup.
'''

# Camera calibration and parameter estimation completed in Question 1
# Intrinsic and extrinsic parameters have been computed
# Rotation angles along each axis have been calculated
# Calibration results are stored and can be used for further processing


In [26]:
#  Importing necessary packages
import numpy as np
import cv2
from scipy.spatial.transform import Rotation
import math
from math import cos, sin, radians, sqrt

In [2]:
# Reading the rotational vectors from saved files from Question 1, Run q2 only after q1 as we have necessary files generated from q1
rot_v = []

with open('images/left/rotat_vector.txt', 'r') as f:
    for line in f :
        rot_v.append([[float(num)] for num in line.split(' ')])

np_rot_v = np.array(rot_v)

print("Rotation vector:")    
print(np_rot_v)

Rotation vector:
[[[-0.06411]
  [ 0.01005]
  [-0.02888]]

 [[-0.08194]
  [-0.00255]
  [-0.03956]]

 [[-0.07244]
  [ 0.00848]
  [-0.05837]]

 [[-0.06368]
  [-0.02992]
  [ 0.00247]]

 [[-0.05461]
  [-0.04569]
  [ 0.03655]]

 [[-0.0284 ]
  [-0.02158]
  [-0.02723]]

 [[-0.00576]
  [-0.01766]
  [-0.04316]]

 [[ 0.00373]
  [-0.02953]
  [-0.03722]]

 [[-0.00557]
  [ 0.00225]
  [-0.03709]]

 [[ 0.00135]
  [ 0.14067]
  [-0.04041]]]


In [3]:
r_obj = Rotation.from_rotvec(np.array(rot_v[0]).reshape(1,3))
rot_matrix = r_obj.as_matrix()

print("ROTATIONAL MATRIX")
print(rot_matrix)

ROTATIONAL MATRIX
[[[ 0.99953267  0.0285337   0.01096691]
  [-0.02917774  0.99752897  0.06391105]
  [-0.00911619 -0.06420117  0.99789534]]]


In [4]:
# Creation of rotation matrix from rotation vector
xc, xs = cos(radians(np_rot_v[0][0][0])), sin(radians(np_rot_v[0][0][0]))
yc, ys = cos(radians(np_rot_v[0][1][0])), sin(radians(np_rot_v[0][1][0]))
zc, zs = cos(radians(np_rot_v[0][2][0])), sin(radians(np_rot_v[0][2][0]))

In [5]:
#Creation of  rotation matrices along x,y and z
rot_x_mtx = np.array([[1,0,0,0],[0,xc,-xs,0],[0,xs,-xc,0],[0,0,0,1]])
rot_y_mtx = np.array([[yc,0,ys,0],[0,1,0,0],[-ys,0,yc,0],[0,0,0,1]])
rot_z_mtx = np.array([[zc,-zs,0,0],[zs,zc,0,0],[0,0,1,0],[0,0,0,1]])

In [6]:
# Reading the translation vectors from saved files

tra_v = []

with open('images/left/trans_vector.txt', 'r') as f:
    for line in f :
        tra_v.append([[float(num)] for num in line.split(' ')])
        
np_tra_v = np.array(tra_v)

print("Transation vector:")    
print(np_tra_v)

Transation vector:
[[[-0.70711]
  [-2.78165]
  [ 8.36554]]

 [[-1.20581]
  [-2.53829]
  [ 8.64232]]

 [[-1.96116]
  [-2.31523]
  [ 8.79577]]

 [[-2.46894]
  [-2.70203]
  [ 8.68045]]

 [[-2.61204]
  [-2.9879 ]
  [ 8.52294]]

 [[-2.75342]
  [-2.35787]
  [ 8.10509]]

 [[-2.54701]
  [-2.23151]
  [ 7.51416]]

 [[-2.43804]
  [-2.17882]
  [ 6.74177]]

 [[-2.65625]
  [-2.51014]
  [ 8.99854]]

 [[-1.89836]
  [-2.98701]
  [11.01494]]]


In [7]:
# Translated matrix
tx = np_tra_v[0][0][0]
ty = np_tra_v[0][1][0]
tz = np_tra_v[0][2][0]
trans_mtx = np.array([[1,0,0,tx],[0,1,0,ty],[0,0,1,tz],[0,0,0,1]])

print("TRANSLATIONAL MATRIX")
print(trans_mtx)

TRANSLATIONAL MATRIX
[[ 1.       0.       0.      -0.70711]
 [ 0.       1.       0.      -2.78165]
 [ 0.       0.       1.       8.36554]
 [ 0.       0.       0.       1.     ]]


In [8]:
# Reading the intrinsic camera matrix from saved files

c_mtx = []
 
with open('images/left/camera_matrix.txt', 'r') as f:
    for line in f :
        c_mtx.append([float(num) for num in line.split(' ')])

print("CAMERA INTRINSIC MATRIX:")
print(c_mtx)

CAMERA INTRINSIC MATRIX:
[[460.5366, 0.0, 309.2375], [0.0, 482.0849, 243.95965], [0.0, 0.0, 1.0]]


In [9]:
# Intrinsic matrix

int_mtx = np.append(np.append(c_mtx, [[0],[0],[1]], axis=1), [np.array([0,0,0,1])], axis=0)
print('Intrinsinc mtx:')
print(int_mtx)

Intrinsinc mtx:
[[460.5366    0.      309.2375    0.     ]
 [  0.      482.0849  243.95965   0.     ]
 [  0.        0.        1.        1.     ]
 [  0.        0.        0.        1.     ]]


In [10]:
# Extrinsic matrix

ext_mtx = np.dot(rot_z_mtx, np.dot(rot_y_mtx, np.dot(rot_x_mtx, trans_mtx)))
print('Extrinsinc mtx')
print(ext_mtx)

Extrinsinc mtx
[[ 9.99999858e-01  5.03854484e-04 -1.74841459e-04 -7.09974089e-01]
 [-5.04051059e-04  9.99999247e-01  1.11901862e-03 -2.77193029e+00]
 [-1.75405589e-04 -1.11893033e-03 -9.99999359e-01 -8.36229813e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
# Camera Matrix

camera_matrix = np.dot(int_mtx, ext_mtx)
print("CAMERA MATRIX: ")
print(camera_matrix)

CAMERA MATRIX: 
[[ 4.60482292e+02 -1.13971788e-01 -3.09317823e+02 -2.91290522e+03]
 [-2.85787290e-01  4.81811563e+02 -2.43420032e+02 -3.37636906e+03]
 [-1.75405589e-04 -1.11893033e-03 -9.99999359e-01 -7.36229813e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [12]:
# Calculate the inverse of the camera matrix to transform from image plane to real-world coordinates
inverse_mat = -np.linalg.inv(camera_matrix) 

# Homogeneous coordinates of two points in the image plane
project_points1 = np.array([[5],[10],[30],[1]])
project_points2 = np.array([[100],[90],[30],[1]])

# Transform points from image plane to real-world coordinates
real_dim_p1 = inverse_mat.dot(project_points1)
real_dim_p2 = inverse_mat.dot(project_points2)


In [46]:
#  this matrix below is the rotational matrix we got earlier

R = np.array([[0.99953267, 0.0285337, 0.01096691],
              [-0.02917774, 0.99752897, 0.06391105],
              [-0.00911619, -0.06420117, 0.99789534]])

# Using cv2.RQDecomp3x3() to decompose the rotation matrix into Euler angles
roll_pitch_yaw = cv2.RQDecomp3x3(R)[0]

# Extracting individual Euler angles
roll = roll_pitch_yaw[0]
pitch = roll_pitch_yaw[1]
yaw = roll_pitch_yaw[2]

# Convert angles from radians to degrees
roll_degrees = np.degrees(roll)
pitch_degrees = np.degrees(pitch)
yaw_degrees = np.degrees(yaw)

print("Roll (X axis rotation):", roll_degrees, "degrees")
print("Pitch (Y axis rotation):", pitch_degrees, "degrees")
print("Yaw (Z axis rotation):", yaw_degrees, "degrees")

Roll (X axis rotation): -210.9138382679699 degrees
Pitch (Y axis rotation): 29.927100873638512 degrees
Yaw (Z axis rotation): -95.8024478478814 degrees
